In [2]:
import pandas as pd
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt

path_to_fDepth     = '/home/gterren/dynamic_update/functional_forecast_dynamic_update/fDepth'
path_to_data       = '/home/gterren/dynamic_update/data'
path_to_validation = '/home/gterren/dynamic_update/validation'

In [15]:
param_name = 'trust_rate'
param = 0.2

df_ = pd.read_csv(path_to_validation + f'/validation_ffc-WIS-{param_name}.csv')
print(df_.columns)

df_ = df_[[param_name, 
           'asset', 
           'lat', 
           'lon', 
           'day', 
           'time',
           'WIS_e', 
           'WIS_f', 
           'RMSE_s', 
           'RMSE_tau']]

df_['x'] = np.cos(np.radians(df_['lat']))*np.cos(np.radians(df_['lon']))
df_['y'] = np.cos(np.radians(df_['lat']))*np.sin(np.radians(df_['lon']))
df_['z'] = np.sin(np.radians(df_['lat']))

X_ = df_.loc[df_[param_name] == param].reset_index(drop=True).sort_values(by=['asset', 
                                                                              'day', 
                                                                              'time'])
X_ = X_[['time', 'WIS_e', 'RMSE_s', 'RMSE_tau']]
# X_ = X_[['x', 
#          'y', 
#          'z', 
#          'time',
#          'WIS_e', 
#          'RMSE_s', 
#          'RMSE_tau']]
print(X_)

idx_ = df_.groupby(['asset', 'time', 'day'])['WIS_f'].idxmin()

y_ = df_.loc[idx_, ['asset', 
                    'day', 
                    'time',
                    param_name]].reset_index(drop=True).sort_values(by=['asset', 
                                                                        'day', 
                                                                        'time'])
y_ = y_[[param_name]]

print(X_.shape, y_.shape)

Index(['forget_rate_f', 'forget_rate_e', 'length_scale_f', 'length_scale_e',
       'lookup_rate', 'trust_rate', 'nu', 'gamma', 'xi', 'kappa_min',
       'kappa_max', 'time', 'asset', 'day', 'lon', 'lat', 'n_scenarios',
       'WIS_e', 'WIS_f', 'RMSE_tau', 'RMSE_s'],
      dtype='object')
       time     WIS_e    RMSE_s  RMSE_tau
0        72  0.028919  0.034660  0.046792
7260    144  0.028661  0.035498  0.050431
14520   216  0.033882  0.049283  0.043375
1        72  0.022118  0.046389  0.007847
7261    144  0.026172  0.063156  0.012700
...     ...       ...       ...       ...
14518   144  0.039079  0.044493  0.144572
21778   216  0.072659  0.140558  0.125585
7259     72  0.028134  0.083433  0.112879
14519   144  0.031030  0.089102  0.080654
21779   216  0.028072  0.034347  0.068316

[21780 rows x 4 columns]
(21780, 4) (21780, 1)


In [ ]:
from sklearn.linear_model import LassoCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_, y_, 
                                                    test_size=0.2, 
                                                    shuffle=True, 
                                                    random_state=42)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# Standardize features (important for Lasso)
_scaler = StandardScaler()
X_train = _scaler.fit_transform(X_train)
X_test  =  _scaler.transform(X_test)

# Create and fit LassoCV model
lasso_cv = LassoCV(alphas=np.logspace(-12, 6, 10),
                   cv=5, 
                   max_iter=10000,
                   fit_intercept=True, 
                   random_state=42).fit(X_train, y_train.to_numpy().ravel())

# Best alpha found
print("Best alpha:", lasso_cv.alpha_)

# Coefficients
print("Lasso coefficients:", lasso_cv.coef_)

# Score on test data
print("R^2 score on test set:", lasso_cv.score(X_test, y_test.to_numpy().ravel()))
print("RMSE:", np.sqrt(mean_squared_error(lasso_cv.predict(X_test), y_test)))
print("MAPE:", mean_absolute_percentage_error(lasso_cv.predict(X_test), y_test))
print(lasso_cv.alpha_)
plt.semilogx(lasso_cv.alphas_, lasso_cv.mse_path_.mean(axis=1), marker='o')
plt.axvline(lasso_cv.alpha_, linestyle='--', color='r', label='Best alpha')
plt.xlabel("Alpha")
plt.ylabel("Mean Squared Error")
plt.title("LassoCV - MSE vs Alpha")
plt.legend()
plt.show()

(17424, 7) (4356, 7) (17424, 1) (4356, 1)


In [16]:
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_, y_, 
                                                    test_size=0.2, 
                                                    shuffle=True, 
                                                    random_state=42)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# Standardize features (important for Ridge)
_scaler = StandardScaler()
X_train = _scaler.fit_transform(X_train)
X_test  =  _scaler.transform(X_test)

ridge_cv = RidgeCV(alphas=np.logspace(-6, 6, 25),
                   cv=5, 
                   fit_intercept=True).fit(X_train, y_train.to_numpy().ravel())

# Best alpha found
print("Best alpha:", ridge_cv.alpha_)

# Coefficients
print("Ridge coefficients:", ridge_cv.coef_)

# Score on test data
print("R^2 score on test set:", ridge_cv.score(X_test, y_test.to_numpy().ravel()))
print("RMSE:", np.sqrt(mean_squared_error(ridge_cv.predict(X_test), y_test)))
print("MAPE:", mean_absolute_percentage_error(ridge_cv.predict(X_test), y_test))

(17424, 4) (4356, 4) (17424, 1) (4356, 1)
Best alpha: 100.0
Ridge coefficients: [ 0.05173047 -0.03029106 -0.02227994 -0.07893199]
R^2 score on test set: 0.08017679594213001
RMSE: 0.3952273481233384
MAPE: 0.9551385924498887


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.tree import DecisionTreeRegressor

# Define hyperparameter ranges
param_grid = {'max_depth': [2, 4, 8, 16],
              'min_samples_split': [2, 4, 8, 16],
              'min_samples_leaf': [2, 4, 8, 16, 32, 64],
              'max_features': [None]}

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_, y_, 
                                                    test_size=0.2, 
                                                    shuffle=True, 
                                                    random_state=42)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# Standardize features (important for Lasso)
_scaler = StandardScaler()
X_train = _scaler.fit_transform(X_train)
X_test  = _scaler.transform(X_test)

# Set up grid search
_grid_search = GridSearchCV(DecisionTreeRegressor(random_state=42), 
                            param_grid, 
                            cv=5, 
                            scoring='r2').fit(X_train, y_train)

# Best parameters
print("Best hyperparameters:", _grid_search.best_params_)
print("Best CV score:", _grid_search.best_score_)
print("R2 score on test set:", _grid_search.score(X_test, y_test.to_numpy().ravel()))
print("RMSE:", np.sqrt(mean_squared_error(_grid_search.predict(X_test), y_test)))
print("MAPE:", mean_absolute_percentage_error(_grid_search.predict(X_test), y_test))

(17424, 7) (4356, 7) (17424, 1) (4356, 1)
Best hyperparameters: {'max_depth': 4, 'max_features': None, 'min_samples_leaf': 32, 'min_samples_split': 2}
Best CV score: 0.018242240629174056
R2 score on test set: 0.01999376659809382
RMSE: 2.4924711332137965
MAPE: 1.0456604887014258
